In [1]:
import numpy as np
import librosa
import librosa.display
import os
from matplotlib import pyplot as plt
import random

In [9]:
# All mfcc data are contained in numpy_datas, which is a list. 
# The each entry of the list is another list consistes of [mfccs, file name]
MFCCs_DATA = "MFCCsData"
numpy_datas = []

dirlist = os.listdir(MFCCs_DATA)
for d in dirlist:
    d = os.path.join(MFCCs_DATA, d)
    datalist = os.listdir(d)
    datalist = [[np.load(os.path.join(d,x)), os.path.join(d,x)] for x in datalist]
    numpy_datas.extend(datalist)

In [10]:
numpy_datas[200][0].shape

(19, 612)

In [11]:
numpy_datas[100][0].shape

(19, 492)

Notice that the size of mfccs for each emotion file is different, need to resize the data into the same size

In [12]:
for i in range(len(numpy_datas)):
    numpy_datas[i][0] = np.resize(numpy_datas[i][0], (19,512))

In [13]:
numpy_datas[200][0].shape

(19, 512)

In [14]:
numpy_datas[100][0].shape

(19, 512)

So now all data has the same shape

Separate the data list according to the label. Change file name to correct label. [1,0,0] is angry, [0,1,0] is happy, [0,0,1] is normal.

In [8]:
# change file name to correct label
angry = []
happy = []
normal = []

for i in range(len(numpy_datas)):
    file_name = numpy_datas[i][1]
    if "angry" in file_name:
        numpy_datas[i][1] = np.array([1,0,0])
        angry.append(numpy_datas[i])
    elif "happy" in file_name:
        numpy_datas[i][1] = np.array([0,1,0])
        happy.append(numpy_datas[i])
    else:
        numpy_datas[i][1] = np.array([0,0,1])
        normal.append(numpy_datas[i])

Now split the Train-Valid-Test data with a ratio of 60%-20%-20%

In [9]:
# first shuffle the list of each emotion
random.shuffle(angry)
random.shuffle(happy)
random.shuffle(normal)

In [10]:
# then select the first 180 in each emotion as train data, first 60 in each emotion as valid data, and first 60 in each emotion as test data
train_data = angry[:int(len(angry)*0.6)] + happy[:int(len(happy)*0.6)] + normal[:int(len(normal)*0.6)]
valid_data = angry[int(len(angry)*0.6):int(len(angry)*(0.6+0.2))] + happy[int(len(happy)*0.6):int(len(happy)*(0.6+0.2))] + normal[int(len(normal)*0.6):int(len(normal)*(0.6+0.2))]
test_data = angry[int(len(angry)*(0.6+0.2)):] + happy[int(len(happy)*(0.6+0.2)):] + normal[int(len(normal)*(0.6+0.2)):]
random.shuffle(train_data)
random.shuffle(valid_data)
random.shuffle(test_data)

In [11]:
train_data

[[array([[  76.73932   ,  -84.60176   ,    3.5487638 , ...,  -11.48938   ,
             2.698151  ,   -9.07002   ],
         [  66.15152   ,  -61.5111    ,    0.6622479 , ...,  -14.289133  ,
             1.4405489 ,   -7.582421  ],
         [  41.68316   ,  -39.65909   ,   -0.19303009, ...,  -17.332321  ,
             0.19360797,   -5.9470716 ],
         ...,
         [-119.23865   ,    4.1396184 ,    6.461754  , ...,   10.139177  ,
           -15.054461  ,   -2.7969618 ],
         [-115.51207   ,    2.9202943 ,   11.89987   , ...,    9.136017  ,
           -16.416092  ,   -8.297977  ],
         [ -95.40622   ,    2.5544581 ,   11.070562  , ...,    3.8107164 ,
           -12.464893  ,   -2.6386194 ]], dtype=float32),
  array([0, 1, 0])],
 [array([[185.4502   ,  79.75186  , -38.815254 , ...,  -6.238064 ,
           -3.0756633, -24.642685 ],
         [180.55887  ,  84.558365 , -14.188961 , ...,  -5.0943413,
           -1.814429 , -16.750774 ],
         [166.28094  , 117.176735 ,   8.0789

In [12]:
# train, valid, test data are all in list consistes of [mfccs, file name]
# want to change the list to two outputs:
# X: vector of mfccs
# Y: vector of labels
def data2vector(data):
    X = []
    Y = []
    for i in range(len(data)):
        X.append(data[i][0])
        Y.append(data[i][1])
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [13]:
X_train, Y_train = data2vector(train_data)

In [14]:
print(X_train.shape)
print(Y_train.shape)

(540, 512, 19)
(540, 3)


In [15]:
X_valid, Y_valid = data2vector(valid_data)
X_test, Y_test = data2vector(test_data)

Now let's do normalization, by the value of train data

In [16]:
X_train_miu = np.average(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)

In [17]:
print(X_train_miu.shape)
print(X_train_std.shape)

(512, 19)
(512, 19)


In [18]:
X_train_norm = (X_train - X_train_miu)/X_train_std
X_valid_norm = (X_valid - X_train_miu)/X_train_std
X_test_norm = (X_test - X_train_miu)/X_train_std